In [ ]:
#searching kg and bhp
#site: https://www.carfolio.com/search/
#search name
#click Sort descending
#choose max_found(2015:2019)
#choose name with the fewest character count from search term
#pick bhp and kg
#append info to python database







In [61]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException   

data = pd.read_excel('C:\\Users\\Patrice\\Desktop\\Classes\\PIR\\Dataset_V1.2.xlsx')
data.head()
carlist=[]

for name in data.DivCar:
    dict_name = {'name':name, 'kg':float(0),'bhp':float(0)}
    carlist.append(dict_name)



In [64]:

#definir le wedriver et la page web qui va nous intéresser

driver = webdriver.Chrome('C:\\Users\\Patrice\\Desktop\\chromedriver.exe') 
driver.get('https://www.carfolio.com/search/');


#pour chaque voiture dans le dictionnaire, on va faire une recherche

for car in carlist:
    driver.get('https://www.carfolio.com/search/')
    in_y1       = driver.find_element_by_xpath("//div[3]/div/div/input")
    in_y2       = driver.find_element_by_xpath("//input[2]")
    in_name     = driver.find_element_by_xpath("//div/div/input")
    search_but  = driver.find_element_by_xpath("//p[29]/input")
    weight      = []
    bhp         = []
    
    
    #on cherche entre 2015 et 2019, et on cherche le nom de la voiture
    
    in_y1.send_keys('2015')
    in_y2.send_keys('2019')
    in_name.send_keys(car.get('name').lower())
    search_but.click()
    time.sleep(1)
    
    
    #cherche s'il y a au moins un resultat, sinon on passe à la prochaine recherche
    
    try:
        sort_year=driver.find_element_by_css_selector("[class^='longlist ui eight cards']")
        print(1)
    except NoSuchElementException:
        continue
        print(0)
    
    
    #si il y a seulement un resultat on peut pas trier par an, ducoup on doit voir si le bouton existe, et puis l'executer
    
    try:
        sort_year=driver.find_element_by_xpath("//a/i").click()
    except NoSuchElementException:
        pass
    
    
    #chercher toute les informations differents qui nous interesse des résultats
    
    page = requests.get(driver.current_url)
    soup = bs(page.text, 'html.parser')
    
    for bhp_result in soup.findAll("abbr", {"title":"brake horsepower"}):
        bhp.append(bhp_result.next_sibling)
    
    for w_result in soup.findAll("span", {"class":"weight"}):
        weight.append(re.sub("\D","",str(w_result)))
    
    
    car.update({'bhp':np.mean(list(np.float_(bhp))), 'kg':np.mean(list(np.float_(weight)))})
    print(car)
    




#time.sleep(5)
#search_box = driver.find_element_by_name('q')
#search_box.send_keys('ChromeDriver')
#search_box.submit()



1
{'name': 'ACURA NSX', 'kg': 1725.0, 'bhp': 373.0}
1
{'name': 'ALFA ROMEO 4C', 'kg': 1087.0, 'bhp': 177.0}
1
{'name': 'ASTON MARTIN VANTAGE V8', 'kg': 1510.0, 'bhp': 328.0}
1
{'name': 'AUDI TT ROADSTER QUATTRO', 'kg': 1570.0, 'bhp': 225.0}
1
{'name': 'BMW Z4 SDRIVE30I', 'kg': 1490.5, 'bhp': 190.0}
1
{'name': 'BUGATTI CHIRON', 'kg': 1995.0, 'bhp': 1103.0}
1
{'name': 'CHEVROLET CORVETTE', 'kg': 1558.7692307692307, 'bhp': 385.4166666666667}
1
{'name': 'CHEVROLET CORVETTE', 'kg': 1558.7692307692307, 'bhp': 385.4166666666667}
1
{'name': 'CHEVROLET CORVETTE', 'kg': 1558.7692307692307, 'bhp': 385.4166666666667}
1
{'name': 'CHEVROLET CORVETTE', 'kg': 1558.7692307692307, 'bhp': 385.4166666666667}
1
{'name': 'CHEVROLET CORVETTE', 'kg': 1558.7692307692307, 'bhp': 385.4166666666667}
1
{'name': 'CHEVROLET CORVETTE', 'kg': 1558.7692307692307, 'bhp': 385.4166666666667}
1
{'name': 'FERRARI 488 GTB', 'kg': 1370.0, 'bhp': 492.0}
1
{'name': 'FERRARI 488 GTB', 'kg': 1370.0, 'bhp': 492.0}
1
{'name': 'FERR

1
{'name': 'AUDI A3 CABRIOLET QUATTRO', 'kg': 1645.0, 'bhp': 151.5}
1
{'name': 'AUDI A3 QUATTRO', 'kg': 1503.076923076923, 'bhp': 137.3846153846154}
1
{'name': 'AUDI A5 QUATTRO', 'kg': 1666.5, 'bhp': 162.0}
1
{'name': 'AUDI RS 3', 'kg': 1670.7142857142858, 'bhp': 288.0}
1
{'name': 'AUDI RS 5', 'kg': 1702.7777777777778, 'bhp': 316.8888888888889}
1
{'name': 'AUDI S3', 'kg': 1543.5714285714287, 'bhp': 223.71428571428572}
1
{'name': 'AUDI S5', 'kg': 1818.75, 'bhp': 255.08333333333334}
1
{'name': 'AUDI S5 CABRIOLET', 'kg': 1932.5, 'bhp': 253.0}
1
{'name': 'AUDI TT COUPE QUATTRO', 'kg': 1644.4444444444443, 'bhp': 292.3333333333333}
1
{'name': 'AUDI TT RS', 'kg': 1581.6666666666667, 'bhp': 290.6666666666667}
1
{'name': 'AUDI TTS COUPE', 'kg': 1467.5, 'bhp': 216.5}
1
{'name': 'BMW 230I CONVERTIBLE', 'kg': 1664.2857142857142, 'bhp': 185.0}
1
{'name': 'BMW 230I COUPE', 'kg': 1532.888888888889, 'bhp': 185.0}
1
{'name': 'BMW 230I COUPE', 'kg': 1532.888888888889, 'bhp': 185.0}
1
{'name': 'BMW 230I 

1
{'name': 'CADILLAC ATS', 'kg': 1619.6842105263158, 'bhp': 246.64}
1
{'name': 'CADILLAC ATS', 'kg': 1619.6842105263158, 'bhp': 246.64}
1
{'name': 'CADILLAC ATS AWD', 'kg': 1639.7272727272727, 'bhp': 226.5}
1
{'name': 'CADILLAC ATS AWD', 'kg': 1639.7272727272727, 'bhp': 226.5}
1
{'name': 'CADILLAC ATS-V', 'kg': 1726.0, 'bhp': 346.0}
1
{'name': 'CADILLAC ATS-V', 'kg': 1726.0, 'bhp': 346.0}
1
{'name': 'CHEVROLET CRUZE', 'kg': 1362.5, 'bhp': 110.3076923076923}
1
{'name': 'CHEVROLET CRUZE', 'kg': 1362.5, 'bhp': 110.3076923076923}
1
{'name': 'CHEVROLET CRUZE', 'kg': 1362.5, 'bhp': 110.3076923076923}
1


C:\Users\Patrice\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Patrice\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'name': 'CHEVROLET CRUZE PREMIER', 'kg': nan, 'bhp': 114.0}
1
{'name': 'GENESIS G70 AWD', 'kg': 1700.0, 'bhp': 200.85714285714286}
1
{'name': 'GENESIS G70 AWD', 'kg': 1700.0, 'bhp': 200.85714285714286}
1
{'name': 'HYUNDAI ACCENT', 'kg': 1154.888888888889, 'bhp': 102.33333333333333}
1
{'name': 'HYUNDAI ACCENT', 'kg': 1154.888888888889, 'bhp': 102.33333333333333}
1
{'name': 'HYUNDAI VELOSTER', 'kg': 1283.388888888889, 'bhp': 133.27777777777777}
1
{'name': 'HYUNDAI VELOSTER', 'kg': 1283.388888888889, 'bhp': 133.27777777777777}
1
{'name': 'HYUNDAI VELOSTER', 'kg': 1283.388888888889, 'bhp': 133.27777777777777}
1
{'name': 'HYUNDAI VELOSTER', 'kg': 1283.388888888889, 'bhp': 133.27777777777777}
1
{'name': 'HYUNDAI VELOSTER N', 'kg': 1283.388888888889, 'bhp': 133.27777777777777}
1
{'name': 'INFINITI QX30', 'kg': 1526.6666666666667, 'bhp': 155.0}
1
{'name': 'INFINITI QX30 AWD', 'kg': 1561.0, 'bhp': 155.0}
1
{'name': 'JAGUAR XE', 'kg': 1613.0, 'bhp': 199.28}
1
{'name': 'JAGUAR XE', 'kg': 1613.0,

1
{'name': 'HYUNDAI SONATA HYBRID', 'kg': 1611.0, 'bhp': 113.8}
1
{'name': 'HYUNDAI SONATA HYBRID SE', 'kg': 1611.0, 'bhp': 113.8}
1
{'name': 'INFINITI Q50', 'kg': 1754.8333333333333, 'bhp': 225.22222222222223}
1
{'name': 'INFINITI Q50', 'kg': 1754.8333333333333, 'bhp': 225.22222222222223}
1
{'name': 'INFINITI Q50 AWD', 'kg': 1791.3333333333333, 'bhp': 225.22222222222223}
1
{'name': 'INFINITI Q50 AWD', 'kg': 1791.3333333333333, 'bhp': 225.22222222222223}
1
{'name': 'INFINITI Q50 AWD RED SPORT', 'kg': 1814.0, 'bhp': 298.0}
1
{'name': 'INFINITI Q50 RED SPORT', 'kg': 1779.5, 'bhp': 298.0}
1
{'name': 'INFINITI Q70', 'kg': 1849.578947368421, 'bhp': 270.89473684210526}
1
{'name': 'INFINITI Q70', 'kg': 1849.578947368421, 'bhp': 270.89473684210526}
1
{'name': 'INFINITI Q70 AWD', 'kg': 1894.375, 'bhp': 279.5}
1
{'name': 'INFINITI Q70 AWD', 'kg': 1894.375, 'bhp': 279.5}
1
{'name': 'JAGUAR XF', 'kg': 1668.2, 'bhp': 167.08}
1
{'name': 'JAGUAR XF', 'kg': 1668.2, 'bhp': 167.08}
1
{'name': 'JAGUAR XF

1
{'name': 'GENESIS G80 AWD', 'kg': 2077.3333333333335, 'bhp': 272.3333333333333}
1
{'name': 'GENESIS G80 AWD', 'kg': 2077.3333333333335, 'bhp': 272.3333333333333}
1
{'name': 'GENESIS G90 AWD', 'kg': 2197.5, 'bhp': 292.5}
1
{'name': 'GENESIS G90 AWD', 'kg': 2197.5, 'bhp': 292.5}
1
{'name': 'HONDA ACCORD', 'kg': 1512.44, 'bhp': 148.04}
1
{'name': 'HONDA ACCORD', 'kg': 1512.44, 'bhp': 148.04}
1
{'name': 'HONDA ACCORD', 'kg': 1512.44, 'bhp': 148.04}
1
{'name': 'HONDA ACCORD', 'kg': 1512.44, 'bhp': 148.04}
1
{'name': 'HONDA ACCORD', 'kg': 1512.44, 'bhp': 148.04}
1
{'name': 'HONDA ACCORD', 'kg': 1512.44, 'bhp': 148.04}
1
{'name': 'HYUNDAI ELANTRA GT', 'kg': 1329.125, 'bhp': 132.0}
1
{'name': 'HYUNDAI ELANTRA GT', 'kg': 1329.125, 'bhp': 132.0}
1
{'name': 'HYUNDAI ELANTRA GT', 'kg': 1329.125, 'bhp': 132.0}
1
{'name': 'HYUNDAI IONIQ', 'kg': nan, 'bhp': 81.66666666666667}
1
{'name': 'HYUNDAI SONATA', 'kg': 1539.2380952380952, 'bhp': 134.32}
1
{'name': 'HYUNDAI SONATA', 'kg': 1539.2380952380952,

1
{'name': 'HONDA CR-V AWD', 'kg': 1587.2307692307693, 'bhp': 139.46153846153845}
1
{'name': 'HONDA CR-V AWD', 'kg': 1587.2307692307693, 'bhp': 139.46153846153845}
1
{'name': 'HONDA PASSPORT AWD', 'kg': 1903.3333333333333, 'bhp': 209.0}
1
{'name': 'HONDA PILOT AWD', 'kg': 1933.0, 'bhp': 209.0}
1
{'name': 'HONDA PILOT AWD', 'kg': 1933.0, 'bhp': 209.0}
1
{'name': 'HYUNDAI SANTA FE AWD', 'kg': 1823.6666666666667, 'bhp': 185.88888888888889}
1
{'name': 'HYUNDAI SANTA FE AWD', 'kg': 1823.6666666666667, 'bhp': 185.88888888888889}
1
{'name': 'HYUNDAI TUCSON AWD', 'kg': 1582.1538461538462, 'bhp': 126.92307692307692}
1
{'name': 'HYUNDAI TUCSON AWD', 'kg': 1582.1538461538462, 'bhp': 126.92307692307692}
1
{'name': 'INFINITI QX50 AWD', 'kg': 1823.0, 'bhp': 242.0}
1
{'name': 'INFINITI QX60 AWD', 'kg': 2089.5, 'bhp': 185.0}
1
{'name': 'JAGUAR F-PACE', 'kg': 1834.5714285714287, 'bhp': 221.57142857142858}
1
{'name': 'JAGUAR F-PACE', 'kg': 1834.5714285714287, 'bhp': 221.57142857142858}
1
{'name': 'JAGUA

In [66]:
donelist = carlist

1263
